## Need to get navo_utils and workshop_utils

In [31]:
# Astropy and requests imports
import urllib.parse
from html import unescape
import requests

from astropy.coordinates import SkyCoord
import astropy.io.fits as fits
import astropy.units as u
from astropy.table import Table

# VO astroquery module
from navo_utils.registry import Registry
from navo_utils.image import Image, ImageColumn
from navo_utils.spectra import Spectra, SpectraColumn
from navo_utils.cone import Cone
from navo_utils.tap import Tap
from navo_utils.utils import astropy_table_from_votable_response

# Utility functions used in the workshop.
import workshop_utils.utils

## Suppress unimportant warnings.
import warnings
warnings.filterwarnings("ignore")

##Make things print nicely
import pprint

## Find AllWISE catalogs available at IRSA

In [18]:
allwise_cone_services = Registry.query(keyword='allwise', service_type='cone', source='irsa')
allwise_cone_services.show_in_notebook()
allwise_cone_services['short_name', 'access_url']

short_name,access_url
str16,str62
AllWISE Metadata,https://irsa.ipac.caltech.edu/SCS?table=allwise_p3as_cdd&amp;
AllWISE Reject,https://irsa.ipac.caltech.edu/SCS?table=allwise_p3as_psr&amp;
AllWISE,https://irsa.ipac.caltech.edu/SCS?table=allwise_p3as_psd&amp;
AllWISE MEP,https://irsa.ipac.caltech.edu/SCS?table=allwise_p3as_mep&amp;
NEOWISE-R ST,https://irsa.ipac.caltech.edu/SCS?table=neowiser_p1bs_psd&amp;


## Get all AllWISE catalog entries within 0.1 deg of your search position

In [24]:
svc = 'https://irsa.ipac.caltech.edu/SCS?table=allwise_p3as_psd'
ra = 9.90704
dec = 8.96507
position = SkyCoord(ra, dec, unit = 'deg')
Sources = Cone.query(service=svc, coords=position, radius=0.1)
Sources

[<Table masked=True length=319>
     designation          ra         dec      sigra   ... pm_qual  cat  load_id
                         deg         deg       arcs   ...                      
        str19          float64     float64   float64  ...   str2  int64  int64 
 ------------------- ----------- ----------- -------- ... ------- ----- -------
 J003939.51+085757.9   9.9146411   8.9661089   0.2781 ...             1    1011
 J003934.82+085727.2   9.8950940   8.9575573   0.4222 ...             1    1011
 J003923.58+090052.1   9.8482680   9.0144995   0.4133 ...             1    1011
 J003920.41+085355.5   9.8350624   8.8987516   0.5749 ...             1    1011
 J003916.74+085633.5   9.8197606   8.9426556   0.4497 ...             1    1011
 J003928.57+090012.7   9.8690550   9.0035282   0.2278 ...             1    1011
 J003946.93+085756.6   9.9455818   8.9657266   0.2767 ...             1    1011
 J003934.54+085429.2   9.8939429   8.9081126   0.1757 ...             1    1011
 J003938

## Find images near your position

In [43]:
svc = 'http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?' # get this info from registry
cds = SkyCoord(185.47873, 4.47365, unit=u.deg) # from astropy.coordinates import SkyCoord
images_for_each_position = Image.query(service=svc, coords=cds, radius=0) #radius=0: image contains point
image_list = images_for_each_position[0]
image_list.show_in_notebook()


idx,sia_title,sia_url,sia_naxes,sia_fmt,sia_ra,sia_dec,sia_naxis,sia_crpix,sia_crval,sia_proj,sia_scale,sia_cd,sia_bp_id,sia_bp_ref,sia_bp_lo,sia_bp_hi,pers_art,glint_art,ordate,hemisphere,scanno,coaddno,ut_date,coadd_key,seesh,magzp,msnr10
,,,,,deg,deg,,pix,deg,,deg / pix,deg / pix,,,,,,,,,,,,,arcs,,
0,2MASS All-Sky Data Release K-Band Atlas Image: 000225 s 088 0080,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/image/ki0880080.fits.gz,2,image/fits,185.48134830000001,4.3938090040000004,[512 1024],[256.5 512.5],[185.4813483 4.393809004],SIN,[-0.0002777777845 0.0002777777845],[-0.000277775673673415 -1.08290212349349e-06 -1.08290212349349e-06 0.000277775673673415],K,2.159e-06,1.8840000000000001e-06,2.4339999999999999e-06,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/pers.tbl,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/glint.tbl,000225,s,88,80,000225,1070542,1.0,19.8719997,14.614
1,2MASS All-Sky Data Release J-Band Atlas Image: 000225 s 088 0080,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/image/ji0880080.fits.gz,2,image/fits,185.48134830000001,4.3938090040000004,[512 1024],[256.5 512.5],[185.4813483 4.393809004],SIN,[-0.0002777777845 0.0002777777845],[-0.000277775673673415 -1.08290212349349e-06 -1.08290212349349e-06 0.000277775673673415],J,1.235e-06,1.066e-06,1.404e-06,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/pers.tbl,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/glint.tbl,000225,s,88,80,000225,1070542,0.99000001000000004,20.739700299999999,16.292
2,2MASS All-Sky Data Release H-Band Atlas Image: 000225 s 088 0080,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/image/hi0880080.fits.gz,2,image/fits,185.48134830000001,4.3938090040000004,[512 1024],[256.5 512.5],[185.4813483 4.393809004],SIN,[-0.0002777777845 0.0002777777845],[-0.000277775673673415 -1.08290212349349e-06 -1.08290212349349e-06 0.000277775673673415],H,1.6619999999999999e-06,1.3999999999999999e-06,1.9240000000000001e-06,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/pers.tbl,https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/glint.tbl,000225,s,88,80,000225,1070542,0.94999998799999996,20.383899700000001,15.601


## Download image.

In [48]:
url = 'https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/image/ki0880080.fits.gz'
out_file_path = workshop_utils.utils.download_file(url)

## Search for a spectrum.

In [62]:
spectral_services = Registry.query(service_type='spectr', source='heasarc', keyword='chanmaster')

spectral_services.show_in_notebook()

idx,waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
0,x-ray,Chandra,ivo://nasa.heasarc/chanmaster,No Description Available,https://heasarc.gsfc.nasa.gov/xamin/vo/ssa?table=chanmaster&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/chanmaster.html,NASA/GSFC HEASARC,simplespectralaccess


In [66]:
svc = spectral_services[0]
svc

waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str5,str7,str29,str24,str64,str58,str17,str20
x-ray,Chandra,ivo://nasa.heasarc/chanmaster,No Description Available,https://heasarc.gsfc.nasa.gov/xamin/vo/ssa?table=chanmaster&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/chanmaster.html,NASA/GSFC HEASARC,simplespectralaccess


In [67]:
position = SkyCoord.from_name('m82')
search_radius = 0.1 # degrees
spectra_for_each_position = Spectra.query(coords = position, radius=search_radius, service = svc)
spectra_for_one_position = spectra_for_each_position[0]
spectra_for_one_position.show_in_notebook()

idx,obsid,status,name,ra,dec,time,detector,grating,exposure,type,pi,public_date,SSA_start_time,SSA_tmid,SSA_stop_time,SSA_duration,SSA_ra,SSA_dec,SSA_coord_obs,SSA_title,SSA_reference,SSA_datalength,SSA_datamodel,SSA_instrument,SSA_publisher,SSA_format,SSA_fov,SSA_wavelength_min,SSA_wavelength_max,SSA_bandwidth,SSA_bandpass
,,,,deg,deg,mjd,,,s,,,mjd,mjd,mjd,mjd,s,deg,deg,deg,,,,,,,,deg,meter,meter,meter,meter
0,17860,archived,M82 X-1,148.95917,69.67972,57639.729200000002,ACIS-S,HETG,51060,TOO,Walton,58010,57639.729200000002,57640.040393247451,57640.350814633915,51060.0,148.95917,69.679720000000003,--,acisf17860N002_pha2,https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/17860/primary/acisf17860N002_pha2.fits.gz,12.0,Spectrum-1.0,ACIS-S,HEASARC,application/fits,0.81103500403976259,1.2397999999999999e-10,6.1991999999999999e-09,6.0752199999999996e-09,3.16159e-09
1,19791,archived,M82 X-1,148.95917,69.67972,57641.570200000002,ACIS-S,HETG,15310,TOO,Walton,58010,57641.570200000002,57641.668022753758,57641.765065583451,15310.0,148.95917,69.679720000000003,--,acisf19791N002_pha2,https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/19791/primary/acisf19791N002_pha2.fits.gz,12.0,Spectrum-1.0,ACIS-S,HEASARC,application/fits,0.81103500403976259,1.2397999999999999e-10,6.1991999999999999e-09,6.0752199999999996e-09,3.16159e-09
2,19792,archived,M82 X-1,148.95917,69.67972,57642.353000000003,ACIS-S,HETG,32440,TOO,Walton,58010,57642.353000000003,57642.558369954859,57642.762945849187,32440.0,148.95917,69.679720000000003,--,acisf19792N002_pha2,https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/19792/primary/acisf19792N002_pha2.fits.gz,12.0,Spectrum-1.0,ACIS-S,HEASARC,application/fits,0.81103500403976259,1.2397999999999999e-10,6.1991999999999999e-09,6.0752199999999996e-09,3.16159e-09


In [69]:
spectra_for_one_position['SSA_reference']

https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/17860/primary/acisf17860N002_pha2.fits.gz
https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/19791/primary/acisf19791N002_pha2.fits.gz
https://heasarc.gsfc.nasa.gov/FTP/chandra/data/science/ao17/cat4/19792/primary/acisf19792N002_pha2.fits.gz


## Download Spectrum.

In [72]:
url = spectra_for_one_position[0]['SSA_reference']
out_file_path = workshop_utils.utils.download_file(url)